# Partie 2 - Chapitre 4 : Les Trois Architectures d'Agents

## Introduction

Maintenant que nous comprenons les outils (chapitre 0, 1, 3) et LangGraph (chapitre 2), plongeons dans le monde **agentique** !

### Qu'est-ce qu'un Agent ?

Un agent est un système qui :
- **Raisonne** sur une tâche
- **Décide** quels outils utiliser
- **Exécute** des actions
- **Adapte** son comportement selon les résultats

### Les Trois Architectures

Dans ce chapitre, nous allons explorer **3 architectures d'agents** différentes, chacune avec ses forces et cas d'usage :

| Architecture | Principe | Cas d'usage |
|--------------|----------|-------------|
| **SimpleToolAgent** | Orchestration directe d'outils | Tâches simples, efficacité |
| **SimpleReActAgent** | Raisonnement explicite (Thought→Action→Observation) | Compréhension du processus, débogage |
| **HierarchicalPlanner** | Décomposition en sous-tâches avec dépendances | Tâches complexes multi-étapes |

### Notre Cas d'Usage : VéloCorp

Nous allons travailler avec le système complet de **VéloCorp**, incluant :
- 📚 **Azure Search** : FAQs, manuels, emails, transcriptions (keyword + vector)
- 🗄️ **SQL Database** : Clients, commandes, produits, stock
- 📞 **CRM API** : Prospects, opportunités, commerciaux, analytics
- 🌐 **Web Search** : Informations marché externes
- 📄 **Report Writer** : Génération de rapports professionnels

**Objectif pédagogique** : Comprendre comment différentes architectures d'agents résolvent les mêmes problèmes en utilisant différentes sources de données.

## Configuration Initiale

Commençons par configurer notre environnement avec TOUS les outils VéloCorp.

In [1]:
# Imports nécessaires
from rag_attack import (
    set_config,
    SimpleToolAgent,
    SimpleReActAgent,
    HierarchicalPlanner,
    # Outils de recherche
    azure_search_tool,
    azure_vector_search_tool,
    web_search_tool,
    # Outils SQL
    sql_query_tool,
    get_database_schema,
    sql_table_info,
    # Outils CRM
    crm_opportunities_tool,
    crm_prospects_tool,
    crm_sales_reps_tool,
    crm_analytics_tool,
    # Outil de rapport
    report_writer_tool
)
import json

print("✅ Imports réussis")
print(f"📦 {9} outils importés")

✅ Imports réussis
📦 9 outils importés


In [ ]:
# Configuration Azure (VéloCorp data)
config = {}


set_config(config)

print("✅ Configuration VéloCorp initialisée")
print(f"🤖 Modèle LLM : {config['chat_deployment']}")

✅ Configuration VéloCorp initialisée
🤖 Modèle LLM : gpt4o


### Vue d'Ensemble des Outils Disponibles

Nous avons **9 outils** couvrant 5 sources de données :

#### 🔍 Recherche Documentaire (Azure Search)
1. **azure_search_tool** : Recherche keyword (BM25) dans les documents
2. **azure_vector_search_tool** : Recherche sémantique par similarité vectorielle

#### 🗄️ Base de Données (SQL)
3. **sql_query_tool** : Exécute des requêtes SQL (T-SQL)
4. **get_database_schema** : Obtient le schéma complet de la BD
5. **sql_table_info** : Info détaillée sur une table spécifique

#### 📞 CRM (API Azure Functions)
6. **crm_opportunities_tool** : Opportunités commerciales
7. **crm_prospects_tool** : Prospects et leads
8. **crm_sales_reps_tool** : Commerciaux et équipes
9. **crm_analytics_tool** : Analytics et KPIs globaux

#### 🌐 Internet
10. **web_search_tool** : Recherche web (DuckDuckGo)

#### 📄 Génération
11. **report_writer_tool** : Génération de rapports formatés

In [7]:
# Créer les listes d'outils par catégorie
search_tools = [azure_search_tool, azure_vector_search_tool]
sql_tools = [sql_query_tool, get_database_schema, sql_table_info]
crm_tools = [crm_opportunities_tool, crm_prospects_tool, crm_sales_reps_tool, crm_analytics_tool]
web_tools = [web_search_tool]
report_tools = [report_writer_tool]

# Liste complète
all_tools = search_tools + sql_tools + crm_tools + web_tools + report_tools

print("🔧 OUTILS DISPONIBLES PAR CATÉGORIE :")
print("="*70)
print(f"\n🔍 Recherche (Azure Search) : {len(search_tools)} outils")
for tool in search_tools:
    print(f"   - {tool.name}")

print(f"\n🗄️  Base de données (SQL)    : {len(sql_tools)} outils")
for tool in sql_tools:
    print(f"   - {tool.name}")

print(f"\n📞 CRM (API)                : {len(crm_tools)} outils")
for tool in crm_tools:
    print(f"   - {tool.name}")

print(f"\n🌐 Web                      : {len(web_tools)} outils")
for tool in web_tools:
    print(f"   - {tool.name}")

print(f"\n📄 Rapports                 : {len(report_tools)} outils")
for tool in report_tools:
    print(f"   - {tool.name}")

print(f"\n{'='*70}")
print(f"📊 TOTAL : {len(all_tools)} outils disponibles")
print("✅ Tous les outils prêts à être utilisés par les agents")

🔧 OUTILS DISPONIBLES PAR CATÉGORIE :

🔍 Recherche (Azure Search) : 2 outils
   - azure_search_tool
   - azure_vector_search_tool

🗄️  Base de données (SQL)    : 3 outils
   - sql_query_tool
   - get_database_schema
   - sql_table_info

📞 CRM (API)                : 4 outils
   - crm_opportunities_tool
   - crm_prospects_tool
   - crm_sales_reps_tool
   - crm_analytics_tool

🌐 Web                      : 1 outils
   - web_search_tool

📄 Rapports                 : 1 outils
   - report_writer_tool

📊 TOTAL : 11 outils disponibles
✅ Tous les outils prêts à être utilisés par les agents


## Questions de Test VéloCorp

Chargeons les questions de test organisées par catégorie et difficulté.

In [8]:
# Charger les questions VéloCorp
with open("tests_tools/sample_data/velocop_questions.json", "r", encoding="utf-8") as f:
    velocop_questions = json.load(f)

print("📋 QUESTIONS VÉLOCORP PAR CATÉGORIE :")
print("="*70)

categories = [
    ("questions_simples", "🟢 Questions Simples (Azure Search)"),
    ("questions_sql_simples", "🟦 Questions SQL Simples"),
    ("questions_api_crm_simples", "📞 Questions CRM Simples"),
    ("questions_moyennes", "🟡 Questions Moyennes"),
    ("questions_sql_moyennes", "🟨 Questions SQL Moyennes"),
    ("questions_api_crm_moyennes", "📱 Questions CRM Moyennes"),
    ("questions_multi_outils", "🔧 Questions Multi-Outils"),
    ("questions_complexes", "🔴 Questions Complexes")
]

for key, label in categories:
    if key in velocop_questions:
        questions = velocop_questions[key]
        print(f"\n{label} ({len(questions)} questions)")
        for i, q in enumerate(questions[:2], 1):  # Show first 2
            print(f"   {i}. {q['question'][:70]}..." if len(q['question']) > 70 else f"   {i}. {q['question']}")
            print(f"      → Outils: {', '.join(q['outils_attendus'])}")

print("\n" + "="*70)

📋 QUESTIONS VÉLOCORP PAR CATÉGORIE :

🟢 Questions Simples (Azure Search) (5 questions)
   1. Quels sont les modèles de vélos électriques proposés par VéloCorp ?
      → Outils: azure_search_tool
   2. Quelle est la durée de garantie des vélos VéloCorp ?
      → Outils: azure_search_tool

🟦 Questions SQL Simples (4 questions)
   1. Quels sont tous les modèles de vélos disponibles dans la base de donné...
      → Outils: sql_query_tool
   2. Quel est le vélo le plus cher dans le catalogue ?
      → Outils: sql_query_tool

📞 Questions CRM Simples (4 questions)
   1. Combien y a-t-il de commerciaux dans l'équipe VéloCorp ?
      → Outils: crm_sales_reps_tool
   2. Combien y a-t-il de prospects qualifiés dans le CRM ?
      → Outils: crm_prospects_tool

🟡 Questions Moyennes (4 questions)
   1. Quels types de freins sont utilisés sur les vélos VéloCorp ?
      → Outils: azure_search_tool
   2. Compare la garantie du cadre et des composants
      → Outils: azure_search_tool

🟨 Questions SQL M

---

# Architecture 1 : SimpleToolAgent

## Le Travailleur Efficace

### Principe

Le **SimpleToolAgent** est l'architecture la plus directe :
1. Le LLM reçoit la question
2. Il décide quels outils appeler
3. Il exécute les outils en séquence
4. Il formule la réponse finale

**Avantages** :
- ✅ Simple et efficace
- ✅ Fiable et prévisible
- ✅ Faible latence
- ✅ Idéal pour la production

**Cas d'usage** :
- Questions directes nécessitant 1-2 outils
- Systèmes de production où la fiabilité est critique
- Applications temps-réel

### Exemple 1 : Recherche Documentaire Simple (Azure Search)

In [9]:
# Créer un SimpleToolAgent avec outils de recherche
search_agent = SimpleToolAgent(
    config=config,
    tools=search_tools,  # azure_search_tool + azure_vector_search_tool
    system_prompt="Tu es un assistant expert sur VéloCorp. Utilise la recherche appropriée (keyword ou vectorielle) selon la question.",
    max_iterations=3,
    verbose=True
)

print("✅ SimpleToolAgent (Recherche) créé")
print(f"🔧 Outils : {[t.name for t in search_agent.tools]}")

✅ SimpleToolAgent (Recherche) créé
🔧 Outils : ['azure_search_tool', 'azure_vector_search_tool']


In [10]:
# Question simple : Recherche keyword
q1 = velocop_questions["questions_simples"][0]

print("❓ QUESTION (Recherche keyword) :")
print(q1["question"])
print(f"\n🔧 Outils attendus : {q1['outils_attendus']}")
print("\n" + "="*70 + "\n")

response1 = search_agent.invoke(q1["question"])

print("\n" + "="*70)
print("💡 RÉPONSE :")
print(response1)

❓ QUESTION (Recherche keyword) :
Quels sont les modèles de vélos électriques proposés par VéloCorp ?

🔧 Outils attendus : ['azure_search_tool']



🚀 SimpleToolAgent Starting
❓ Question: Quels sont les modèles de vélos électriques proposés par VéloCorp ?
🔧 Available tools: azure_search_tool, azure_vector_search_tool
🔄 Max iterations: 3

🤖 Agent thinking... (Tool calls so far: 0/3)
🔧 Agent wants to use tools: ['azure_search_tool']

🤖 Agent thinking... (Tool calls so far: 1/3)
✅ Agent provided final answer

✨ Final Answer Generated


💡 RÉPONSE :
VéloCorp propose plusieurs modèles de vélos électriques, notamment :

1. **Urbain-Confort** : Idéal pour les déplacements quotidiens en ville, offrant confort et fiabilité.
2. **E-Cargo** : Conçu pour les déplacements urbains, offrant également confort et fiabilité.
3. **Sport-Pro** : Adapté pour les déplacements quotidiens en ville, avec un excellent confort et une grande fiabilité.

Ces modèles semblent couvrir différents besoins, allant des dép

### Exemple 2 : Requête SQL Simple

In [11]:
# Créer un agent avec outils SQL
sql_agent = SimpleToolAgent(
    config=config,
    tools=sql_tools,
    system_prompt="Tu es un assistant SQL expert. IMPORTANT: Utilise la syntaxe T-SQL (SQL Server), pas PostgreSQL. Utilise TOP N au lieu de LIMIT N.",
    max_iterations=3,
    verbose=True
)

print("✅ SimpleToolAgent (SQL) créé")
print(f"🔧 Outils : {[t.name for t in sql_agent.tools]}")

✅ SimpleToolAgent (SQL) créé
🔧 Outils : ['sql_query_tool', 'get_database_schema', 'sql_table_info']


In [12]:
# Question SQL simple
q2 = velocop_questions["questions_sql_simples"][1]  # Vélo le plus cher

print("❓ QUESTION (SQL) :")
print(q2["question"])
print(f"\n💡 Note : {q2['note']}")
print("\n" + "="*70 + "\n")

response2 = sql_agent.invoke(q2["question"])

print("\n" + "="*70)
print("💡 RÉPONSE :")
print(response2)

❓ QUESTION (SQL) :
Quel est le vélo le plus cher dans le catalogue ?

💡 Note : E-Sport à 3188.83€



🚀 SimpleToolAgent Starting
❓ Question: Quel est le vélo le plus cher dans le catalogue ?
🔧 Available tools: sql_query_tool, get_database_schema, sql_table_info
🔄 Max iterations: 3

🤖 Agent thinking... (Tool calls so far: 0/3)
🔧 Agent wants to use tools: ['get_database_schema']


/Users/raphaelcousin/rag_attack/rag_attack/tools/sql_tool.py:110: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(schema_query, conn)



🤖 Agent thinking... (Tool calls so far: 1/3)
🔧 Agent wants to use tools: ['sql_query_tool']

🤖 Agent thinking... (Tool calls so far: 2/3)


/Users/raphaelcousin/rag_attack/rag_attack/tools/sql_tool.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✅ Agent provided final answer

✨ Final Answer Generated


💡 RÉPONSE :
Le vélo le plus cher dans le catalogue est le **VéloCorp E-Sport** de la ligne de produits **Electrique**. Voici ses détails :

- **Modèle**: E-Sport
- **Nom**: VéloCorp E-Sport
- **Description**: Le E-Sport révolutionne votre mobilité avec sa technologie électrique avancée.
- **Prix**: 3188.83 €
- **Coût**: 2081.59 €
- **Quantité en stock**: 49
- **Marché cible**: B2C
- **Options de couleur**: gris carbone, blanc nacré, noir mat
- **Options de taille**: S, M, L, XL
- **Poids**: 18.3 kg
- **Matériau**: titane
- **Garantie**: 36 mois
- **Date de lancement**: 3 juin 2024
- **Date de création**: 23 novembre 2025


### Exemple 3 : Requête CRM (API)

In [13]:
# Créer un agent avec outils CRM
crm_agent = SimpleToolAgent(
    config=config,
    tools=crm_tools,
    system_prompt="Tu es un assistant CRM expert pour VéloCorp. Utilise les outils appropriés pour répondre aux questions commerciales.",
    max_iterations=3,
    verbose=True
)

print("✅ SimpleToolAgent (CRM) créé")
print(f"🔧 Outils : {[t.name for t in crm_agent.tools]}")

✅ SimpleToolAgent (CRM) créé
🔧 Outils : ['crm_opportunities_tool', 'crm_prospects_tool', 'crm_sales_reps_tool', 'crm_analytics_tool']


In [14]:
# Question CRM simple
q3 = velocop_questions["questions_api_crm_simples"][2]  # Valeur pipeline

print("❓ QUESTION (CRM) :")
print(q3["question"])
print(f"\n💡 Réponse attendue : {q3['note']}")
print("\n" + "="*70 + "\n")

response3 = crm_agent.invoke(q3["question"])

print("\n" + "="*70)
print("💡 RÉPONSE :")
print(response3)

❓ QUESTION (CRM) :
Quelle est la valeur totale du pipeline d'opportunités ?

💡 Réponse attendue : Environ 57K€ de pipeline value



🚀 SimpleToolAgent Starting
❓ Question: Quelle est la valeur totale du pipeline d'opportunités ?
🔧 Available tools: crm_opportunities_tool, crm_prospects_tool, crm_sales_reps_tool, crm_analytics_tool
🔄 Max iterations: 3

🤖 Agent thinking... (Tool calls so far: 0/3)
🔧 Agent wants to use tools: ['crm_analytics_tool']

🤖 Agent thinking... (Tool calls so far: 1/3)
✅ Agent provided final answer

✨ Final Answer Generated


💡 RÉPONSE :
La valeur totale du pipeline d'opportunités est de **57 585,65 euros**.


### Exemple 4 : Multi-Outils (SQL + Search)

In [15]:
# Agent avec SQL + Search
multi_agent = SimpleToolAgent(
    config=config,
    tools=sql_tools + search_tools,
    system_prompt="Tu es un assistant polyvalent VéloCorp. Utilise SQL pour données structurées, Azure Search pour documentation.",
    max_iterations=5,
    verbose=True
)

print("✅ SimpleToolAgent (Multi-outils) créé")
print(f"🔧 {len(multi_agent.tools)} outils disponibles")

✅ SimpleToolAgent (Multi-outils) créé
🔧 5 outils disponibles


In [16]:
# Question multi-outils
q4 = velocop_questions["questions_multi_outils"][4]  # Stock + specs

print("❓ QUESTION (Multi-outils) :")
print(q4["question"])
print(f"\n💡 Note : {q4['note']}")
print("\n" + "="*70 + "\n")

response4 = multi_agent.invoke(q4["question"])

print("\n" + "="*70)
print("💡 RÉPONSE :")
print(response4)

❓ QUESTION (Multi-outils) :
Quels sont les modèles en rupture de stock et trouve leurs spécifications techniques

💡 Note : SQL pour stock faible, puis Azure Search pour specs techniques



🚀 SimpleToolAgent Starting
❓ Question: Quels sont les modèles en rupture de stock et trouve leurs spécifications techniques
🔧 Available tools: sql_query_tool, get_database_schema, sql_table_info, azure_search_tool, azure_vector_search_tool
🔄 Max iterations: 5

🤖 Agent thinking... (Tool calls so far: 0/5)
🔧 Agent wants to use tools: ['sql_query_tool']


/Users/raphaelcousin/rag_attack/rag_attack/tools/sql_tool.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)



🤖 Agent thinking... (Tool calls so far: 1/5)
🔧 Agent wants to use tools: ['get_database_schema']


/Users/raphaelcousin/rag_attack/rag_attack/tools/sql_tool.py:110: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(schema_query, conn)



🤖 Agent thinking... (Tool calls so far: 2/5)
🔧 Agent wants to use tools: ['sql_query_tool']


/Users/raphaelcousin/rag_attack/rag_attack/tools/sql_tool.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)



🤖 Agent thinking... (Tool calls so far: 3/5)
✅ Agent provided final answer

✨ Final Answer Generated


💡 RÉPONSE :
Il n'y a actuellement aucun modèle en rupture de stock.


### Exemple 5 : Agent Complet (Tous les outils)

In [17]:
# Agent avec TOUS les outils
complete_agent = SimpleToolAgent(
    config=config,
    tools=all_tools,
    system_prompt="""Tu es l'assistant VéloCorp complet avec accès à toutes les sources:
    - Azure Search (keyword/vector) pour documentation
    - SQL pour données transactionnelles (USE T-SQL SYNTAX!)
    - CRM pour données commerciales
    - Web pour informations externes
    - Report Writer pour rapports professionnels
    
    Choisis les outils appropriés selon la question.""",
    max_iterations=10,
    verbose=True
)

print("✅ SimpleToolAgent COMPLET créé")
print(f"🔧 {len(complete_agent.tools)} outils disponibles")

✅ SimpleToolAgent COMPLET créé
🔧 11 outils disponibles


In [18]:
# Question complexe nécessitant plusieurs sources
question_complete = """Analyse complète du modèle E-City :
1. Prix et disponibilité en stock (SQL)
2. Spécifications techniques (Azure Search)
3. Opportunités commerciales en cours (CRM)
"""

print("❓ QUESTION (Agent complet) :")
print(question_complete)
print("\n" + "="*70 + "\n")

response_complete = complete_agent.invoke(question_complete)

print("\n" + "="*70)
print("💡 RÉPONSE :")
print(response_complete)

❓ QUESTION (Agent complet) :
Analyse complète du modèle E-City :
1. Prix et disponibilité en stock (SQL)
2. Spécifications techniques (Azure Search)
3. Opportunités commerciales en cours (CRM)




🚀 SimpleToolAgent Starting
❓ Question: Analyse complète du modèle E-City :
1. Prix et disponibilité en stock (SQL)
2. Spécifications techniques (Azure Search)
3. Opportunités commerciales en cours (CRM)

🔧 Available tools: azure_search_tool, azure_vector_search_tool, sql_query_tool, get_database_schema, sql_table_info, crm_opportunities_tool, crm_prospects_tool, crm_sales_reps_tool, crm_analytics_tool, web_search_tool, report_writer_tool
🔄 Max iterations: 10

🤖 Agent thinking... (Tool calls so far: 0/10)
🔧 Agent wants to use tools: ['sql_query_tool', 'azure_search_tool', 'crm_opportunities_tool']


/Users/raphaelcousin/rag_attack/rag_attack/tools/sql_tool.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)



🤖 Agent thinking... (Tool calls so far: 1/10)
✅ Agent provided final answer

✨ Final Answer Generated


💡 RÉPONSE :
### Analyse complète du modèle E-City

#### 1. Prix et disponibilité en stock
Il y a eu une erreur lors de l'exécution de la requête SQL. La colonne `stock_availability` n'existe pas dans la table `products`. Il est possible que la colonne correcte soit `availability` ou une autre variante. Veuillez vérifier la structure de la table.

#### 2. Spécifications techniques
Il y a eu une erreur lors de la recherche dans Azure Cognitive Search. Le champ `specifications` n'est pas reconnu. Il est possible que les spécifications techniques soient stockées sous un autre champ. Veuillez vérifier les champs disponibles pour la recherche.

#### 3. Opportunités commerciales en cours
Les opportunités commerciales en cours pour le modèle E-City sont les suivantes :

- **Valeur du pipeline**: 57,585.64 €
- **Valeur totale des opportunités**: 213,843.58 €
- **Valeur gagnée**: 18,897.10 €


---

# Architecture 2 : SimpleReActAgent

## Le Penseur Transparent

### Principe

Le **SimpleReActAgent** suit le pattern **ReAct** (Reasoning and Acting) :

```
Boucle pour chaque itération :
  1. THOUGHT (Pensée)   : "Je dois chercher les infos sur les freins"
  2. ACTION (Action)    : Utilise azure_search_tool avec query="freins E-City"
  3. OBSERVATION (Obs.) : "J'ai trouvé 3 documents sur les freins"
  → Répéter jusqu'à avoir assez d'informations
  4. ANSWER (Réponse)   : Formuler la réponse finale
```

**Avantages** :
- ✅ Raisonnement explicite et traçable
- ✅ Excellent pour comprendre le processus de décision
- ✅ Utile pour le débogage
- ✅ Pédagogique - on voit "comment" l'agent pense

### Exemple 1 : Raisonnement avec SQL

In [19]:
# Créer un SimpleReActAgent avec SQL
react_sql_agent = SimpleReActAgent(
    tools=sql_tools,
    max_iterations=5,
    verbose=True
)

print("✅ SimpleReActAgent (SQL) créé")
print("💡 Mode verbose: Vous verrez tout le raisonnement\n")

✅ SimpleReActAgent (SQL) créé
💡 Mode verbose: Vous verrez tout le raisonnement



In [20]:
# Question SQL moyenne nécessitant du raisonnement
q5 = velocop_questions["questions_sql_moyennes"][0]  # Top 5 clients

print("❓ QUESTION (ReAct + SQL) :")
print(q5["question"])
print(f"\n💡 Indice : {q5['note']}")
print("\n" + "="*70 + "\n")

response5 = react_sql_agent.invoke(q5["question"])

print("\n" + "="*70)
print("✨ RÉPONSE FINALE :")
print(response5)

❓ QUESTION (ReAct + SQL) :
Quels sont les 5 clients qui ont le plus dépensé ?

💡 Indice : Utiliser ORDER BY total_spent DESC et TOP 5



🤖 SimpleReAct Agent
❓ Question: Quels sont les 5 clients qui ont le plus dépensé ?
🔧 Tools: sql_query_tool, get_database_schema, sql_table_info
🔄 Max iterations: 5


──────────────────────────────────────────────────────────────────────
🔄 ITERATION 1/5
──────────────────────────────────────────────────────────────────────

💭 THOUGHT:
   Pour répondre à la question "Quels sont les 5 clients qui ont le plus dépensé ?", je dois d'abord identifier les tables pertinentes dans la base de données qui contiennent des informations sur les clients et leurs dépenses. 

Étape 1 : Obtenir le schéma de la base de données pour identifier les tables pertinentes.

Je vais utiliser l'outil `get_database_schema` pour obtenir les informations sur le schéma de la base de données.

Query: get_database_schema

❌ ACTION ERROR:
   Tool 'get_database_schema' not found

───────

### Exemple 2 : Raisonnement avec CRM

In [ ]:
# ReAct avec CRM
react_crm_agent = SimpleReActAgent(
    tools=crm_tools,
    max_iterations=4,
    verbose=True
)

print("✅ SimpleReActAgent (CRM) créé\n")

In [ ]:
# Question CRM moyenne
q6 = velocop_questions["questions_api_crm_moyennes"][2]  # Meilleur commercial

print("❓ QUESTION (ReAct + CRM) :")
print(q6["question"])
print(f"\n💡 Note : {q6['note']}")
print("\n" + "="*70 + "\n")

response6 = react_crm_agent.invoke(q6["question"])

print("\n" + "="*70)
print("✨ RÉPONSE FINALE :")
print(response6)

### Exemple 3 : ReAct avec Multi-Outils

In [ ]:
# ReAct avec tous les outils
react_complete = SimpleReActAgent(
    tools=all_tools,
    max_iterations=8,
    verbose=True
)

print("✅ SimpleReActAgent COMPLET créé")
print(f"🔧 {len(all_tools)} outils disponibles\n")

In [ ]:
# Question multi-outils complexe
q7 = velocop_questions["questions_multi_outils"][3]  # Prix E-City + comparaison web

print("❓ QUESTION (ReAct Multi-outils) :")
print(q7["question"])
print(f"\n💡 Note : {q7['note']}")
print("\n" + "="*70 + "\n")

response7 = react_complete.invoke(q7["question"])

print("\n" + "="*70)
print("✨ RÉPONSE FINALE :")
print(response7)

### Observation du Pattern ReAct

**Regardez attentivement** la sortie ci-dessus. Vous devriez voir le cycle complet :

```
ITERATION 1/N
─────────────
💭 THOUGHT: Je dois d'abord...
🔧 ACTION: sql_query_tool(query="...")
👁️ OBSERVATION: Found 1 result...

ITERATION 2/N
─────────────
💭 THOUGHT: Maintenant je dois...
🔧 ACTION: web_search_tool(...)
👁️ OBSERVATION: ...
```

Ce pattern montre **comment** l'agent raisonne pour arriver à sa conclusion.

---

# Architecture 3 : HierarchicalPlanner

## L'Organisateur Stratégique

### Principe

Le **HierarchicalPlanner** décompose les tâches complexes en sous-tâches avec gestion des dépendances :

```
1. ANALYSE : Comprendre l'objectif global
2. PLANIFICATION : Créer un plan avec des étapes et dépendances
   - Step 1: Chercher les infos X
   - Step 2: Chercher les infos Y (dépend de Step 1)
   - Step 3: Synthétiser (dépend de Step 1 et 2)
3. VALIDATION : Vérifier que le plan est cohérent
4. EXÉCUTION : Exécuter les étapes dans l'ordre correct
5. FINALISATION : Synthétiser les résultats
```

**Avantages** :
- ✅ Gère des tâches multi-étapes complexes
- ✅ Gestion explicite des dépendances
- ✅ Excellent pour des workflows d'affaires

### Exemple 1 : Analyse Multi-Sources

In [ ]:
# Créer un HierarchicalPlanner avec plusieurs sources
planner_multi = HierarchicalPlanner(
    tools=sql_tools + search_tools + crm_tools,
    max_steps=10
)

print("✅ HierarchicalPlanner (Multi-sources) créé")
print(f"🔧 {len(planner_multi.tools)} outils disponibles")
print(f"📊 Max steps : {planner_multi.max_steps}\n")

In [ ]:
# Objectif complexe nécessitant une planification
objectif1 = """Rapport complet pour le conseil d'administration :
1. Identifier les 3 produits avec le stock le plus faible (SQL)
2. Trouver les specs techniques de ces produits (Azure Search)
3. Vérifier s'il y a des opportunités commerciales en cours pour ces produits (CRM)
4. Synthétiser les risques et recommandations
"""

print("🎯 OBJECTIF COMPLEXE :")
print(objectif1)
print("\n" + "="*70 + "\n")

result1 = planner_multi.execute(objectif1)

print("\n" + "="*70)
print("📋 PLAN CRÉÉ PAR LE PLANNER")
print("="*70)

for step in result1['plan']:
    status_emoji = {
        "pending": "⏳",
        "in_progress": "🔄",
        "completed": "✅",
        "failed": "❌"
    }.get(step['status'], "❓")
    
    print(f"\n{status_emoji} Step {step['step_id']}: {step['description']}")
    if step['dependencies']:
        print(f"   Dépendances: {step['dependencies']}")
    if step['tool']:
        print(f"   Outil: {step['tool']}")

In [ ]:
# Afficher le résultat final
print("\n" + "="*70)
print("📊 RÉSULTAT FINAL SYNTHÉTISÉ")
print("="*70)
print(result1['final_result'])

print("\n" + "="*70)
print("📈 STATISTIQUES D'EXÉCUTION")
print("="*70)
print(f"Nombre total d'étapes : {len(result1['plan'])}")
print(f"Étapes complétées : {sum(1 for s in result1['plan'] if s['status'] == 'completed')}")
print(f"Outils utilisés : {len(set(s['tool'] for s in result1['plan'] if s['tool']))}")

### Exemple 2 : Génération de Rapport Complexe

In [ ]:
# Planner avec tous les outils incluant report writer
planner_complete = HierarchicalPlanner(
    tools=all_tools,
    max_steps=12
)

print("✅ HierarchicalPlanner COMPLET créé")
print(f"🔧 {len(all_tools)} outils disponibles\n")

In [ ]:
# Question complexe avec génération de rapport
q8 = velocop_questions["questions_complexes"][2]  # Rapport conseil d'administration

print("🎯 OBJECTIF COMPLEXE (avec rapport) :")
print(q8["question"])
print(f"\n🔧 Outils attendus : {', '.join(q8['outils_attendus'])}")
print("\n" + "="*70 + "\n")

result2 = planner_complete.execute(q8["question"])

print("\n" + "="*70)
print("📋 PLAN D'EXÉCUTION")
print("="*70)

for step in result2['plan']:
    status = "✅" if step['status'] == 'completed' else "⏳"
    print(f"{status} Step {step['step_id']}: {step['description'][:80]}")
    if step['tool']:
        print(f"   └─ Outil: {step['tool']}")

print("\n" + "="*70)
print("📄 RAPPORT FINAL")
print("="*70)
print(result2['final_result'])

---

# Comparaison des Trois Architectures

## Test Comparatif sur la Même Question

In [ ]:
# Question de benchmark
question_benchmark = """Analyse du modèle E-City :
- Prix et stock (SQL)
- Caractéristiques techniques (Azure Search)
- Y a-t-il des opportunités commerciales actives ? (CRM)
"""

print("🔬 TEST COMPARATIF DES 3 ARCHITECTURES")
print("="*80)
print(f"\n❓ Question: {question_benchmark}\n")

import time

# Test SimpleToolAgent
print("\n" + "─"*80)
print("1️⃣  SIMPLETOOLAGENT")
print("─"*80)

agent_test1 = SimpleToolAgent(
    config=config,
    tools=sql_tools + search_tools + crm_tools,
    max_iterations=5,
    verbose=False
)

start1 = time.time()
resp1 = agent_test1.invoke(question_benchmark)
time1 = time.time() - start1

print(f"⏱️  Temps: {time1:.2f}s")
print(f"💡 Réponse: {resp1[:200]}...\n")

# Test SimpleReActAgent
print("\n" + "─"*80)
print("2️⃣  SIMPLEREACTAGENT")
print("─"*80)

agent_test2 = SimpleReActAgent(
    tools=sql_tools + search_tools + crm_tools,
    max_iterations=5,
    verbose=False
)

start2 = time.time()
resp2 = agent_test2.invoke(question_benchmark)
time2 = time.time() - start2

print(f"⏱️  Temps: {time2:.2f}s")
print(f"💡 Réponse: {resp2[:200]}...\n")

# Test HierarchicalPlanner
print("\n" + "─"*80)
print("3️⃣  HIERARCHICALPLANNER")
print("─"*80)

agent_test3 = HierarchicalPlanner(
    tools=sql_tools + search_tools + crm_tools,
    max_steps=8
)

start3 = time.time()
result3 = agent_test3.execute(question_benchmark)
time3 = time.time() - start3
resp3 = result3['final_result']

print(f"⏱️  Temps: {time3:.2f}s")
print(f"📋 Plan steps: {len(result3['plan'])}")
print(f"💡 Réponse: {resp3[:200]}...\n")

# Résumé
print("\n" + "="*80)
print("📊 RÉSUMÉ COMPARATIF")
print("="*80)
print(f"\nSimpleToolAgent      : {time1:.2f}s - Direct et efficace")
print(f"SimpleReActAgent     : {time2:.2f}s - Raisonnement explicite")
print(f"HierarchicalPlanner  : {time3:.2f}s - Planification structurée")

fastest = min(time1, time2, time3)
if fastest == time1:
    print("\n🏆 SimpleToolAgent est le plus rapide")
elif fastest == time2:
    print("\n🏆 SimpleReActAgent est le plus rapide")
else:
    print("\n🏆 HierarchicalPlanner est le plus rapide")

## Tableau Comparatif Détaillé

In [ ]:
import pandas as pd

comparaison = pd.DataFrame({
    "Critère": [
        "Complexité",
        "Transparence",
        "Fiabilité",
        "Vitesse",
        "Gestion dépendances",
        "Multi-sources",
        "Cas d'usage idéal"
    ],
    "SimpleToolAgent": [
        "Faible ⭐",
        "Moyenne ⭐⭐",
        "Excellente ⭐⭐⭐",
        "Rapide ⭐⭐⭐",
        "Non",
        "Oui ✅",
        "Questions directes, production"
    ],
    "SimpleReActAgent": [
        "Moyenne ⭐⭐",
        "Haute ⭐⭐⭐",
        "Excellente ⭐⭐⭐",
        "Moyenne ⭐⭐",
        "Non",
        "Oui ✅",
        "Débogage, compréhension"
    ],
    "HierarchicalPlanner": [
        "Haute ⭐⭐⭐",
        "Haute ⭐⭐⭐",
        "Excellente ⭐⭐⭐",
        "Plus lent ⭐",
        "Oui ✅",
        "Oui ✅",
        "Analyses complexes, workflows"
    ]
})

print("\n📊 COMPARAISON DES TROIS ARCHITECTURES")
print("="*80)
print(comparaison.to_string(index=False))
print("="*80)

---

# Exercices Pratiques

## Exercice 1 : Tester Différentes Questions

Choisissez des questions de la base VéloCorp et testez-les avec les 3 architectures.

In [ ]:
# À VOUS : Choisissez une question et testez

# Catégories disponibles:
# - questions_simples
# - questions_sql_simples / questions_sql_moyennes
# - questions_api_crm_simples / questions_api_crm_moyennes
# - questions_multi_outils
# - questions_complexes

ma_question = velocop_questions["questions_multi_outils"][0]

print(f"❓ Question choisie: {ma_question['question']}")
print(f"🔧 Outils attendus: {ma_question['outils_attendus']}")

# TODO: Créez un agent et testez
# mon_agent = ...
# reponse = mon_agent.invoke(ma_question['question'])

## Exercice 2 : Créer un Workflow Personnalisé

Créez votre propre question complexe qui nécessite plusieurs sources.

In [ ]:
# À VOUS : Créez votre propre workflow

ma_question_custom = """TODO: Écrivez votre question ici.
Exemple: Identifie les clients B2B qui ont dépensé plus de 10000€,
trouve s'il y a des opportunités ouvertes pour eux dans le CRM,
et génère un rapport de suivi commercial.
"""

# TODO: Choisissez l'architecture appropriée
# Quelle architecture est la meilleure pour votre question ?
# - SimpleToolAgent pour efficacité ?
# - SimpleReActAgent pour voir le raisonnement ?
# - HierarchicalPlanner pour la planification ?

# TODO: Créez l'agent et testez
# mon_agent_custom = ...
# resultat_custom = ...

## Exercice 3 : Analyse Comparative

Pour une question complexe, comparez :
- Temps d'exécution
- Nombre d'appels d'outils
- Qualité de la réponse
- Facilité de débogage

In [ ]:
# À VOUS : Analyse comparative détaillée

question_analyse = velocop_questions["questions_complexes"][0]["question"]

# TODO: Testez avec les 3 architectures
# TODO: Notez vos observations
# TODO: Quelle architecture préférez-vous et pourquoi ?

print("Mes observations:")
print("1. SimpleToolAgent: ...")
print("2. SimpleReActAgent: ...")
print("3. HierarchicalPlanner: ...")
print("\nConclusion: ...")

---

# Conclusion

## Ce que Vous Avez Appris

✅ **Trois architectures d'agents différentes** avec des forces complémentaires

✅ **11 outils couvrant 5 sources de données** :
- Azure Search (keyword + vector)
- SQL Database
- CRM API
- Web Search
- Report Writer

✅ **Quand utiliser chaque architecture** selon vos besoins

✅ **Comment combiner plusieurs sources** pour répondre à des questions complexes

✅ **L'importance du choix des outils** selon le type de données

## Principes Clés

1. **Pas d'architecture "meilleure"** - seulement des architectures adaptées à différents cas

2. **La transparence a un coût** - ReAct et Planner sont plus lents mais plus explicables

3. **Les bons outils font la différence** - Utiliser SQL pour données structurées, Search pour docs, CRM pour commercial

4. **La planification aide la complexité** - HierarchicalPlanner excelle pour multi-étapes

5. **Toujours tester et comparer** - Ce qui fonctionne pour un cas peut ne pas fonctionner pour un autre

## Prochaines Étapes

Vous êtes maintenant prêt à :
- Créer vos propres agents avec les outils appropriés
- Choisir l'architecture adaptée à votre cas d'usage
- Combiner plusieurs sources de données efficacement
- Déboguer et optimiser vos agents

**Félicitations ! Vous maîtrisez maintenant les systèmes agentiques multi-sources ! 🎉**